In [4]:
import optuna
import pandas as pd
import numpy as np
import gc
import time
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder, MinMaxScaler, RobustScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import roc_auc_score
import xgboost as xgb

# Paths to datasets
train_path = r"C:\Users\paulo\OneDrive\Documents\kaggle_competition_2_datasets\train.csv"

def import_data(path, index_col=None):
    """Import data from a CSV file and optimize memory usage."""
    start_time = time.time()
    df = pd.read_csv(path, index_col=index_col)
    df = reduce_mem_usage(df)
    print(f"Data imported and memory optimized in {time.time() - start_time:.2f} seconds")
    print(f"Data shape: {df.shape}, Memory usage: {df.memory_usage().sum() / 1024**2:.2f} MB")
    return df

def reduce_mem_usage(df):
    """Iterate through all the columns of a dataframe and modify the data type to reduce memory usage."""
    start_mem = df.memory_usage().sum() / 1024**2
    print(f"Initial memory usage: {start_mem:.2f} MB")
    for col in df.columns:
        col_type = df[col].dtype
        if isinstance(col_type, pd.IntervalDtype):
            continue
        if pd.api.types.is_integer_dtype(col_type):
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                df[col] = df[col].astype(np.int8)
            elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                df[col] = df[col].astype(np.int16)
            elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                df[col] = df[col].astype(np.int32)
            elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                df[col] = df[col].astype(np.int64)  
        elif pd.api.types.is_float_dtype(col_type):
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                df[col] = df[col].astype(np.float32)
            else:
                df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    print(f"Memory usage after optimization: {end_mem:.2f} MB")
    return df

def feature_engineering(df):
    """Feature engineering on the dataset."""
    start_time = time.time()
    # Binning age and converting to categorical labels instead of intervals
    df.loc[:, 'Age_Type'] = pd.cut(df['Age'], bins=7, labels=False)
    df.loc[:, 'Vehicle_Age'] = pd.Categorical(df['Vehicle_Age']).codes
    df.loc[:, 'Vehicle_Damage'] = pd.Categorical(df['Vehicle_Damage']).codes
    df.loc[:, 'Previously_Insured'] = pd.Categorical(df['Previously_Insured']).codes

    df.loc[:, 'Age_x_Vehicle_Age'] = df['Age_Type'] * df['Vehicle_Age']
    df.loc[:, 'Age_x_Vehicle_Damage'] = df['Age_Type'] * df['Vehicle_Damage']
    df.loc[:, 'Age_x_Previously_Insured'] = df['Age_Type'] * df['Previously_Insured']

    fac_pre = ['Policy_Sales_Channel', 'Vehicle_Damage', 'Annual_Premium', 'Vintage', 'Age_Type']
    col_pre = []
    for i in fac_pre:
        df.loc[:, 'Previously_Insured_x_' + i] = pd.factorize(df['Previously_Insured'].astype(str) + df[i].astype(str))[0]
        col_pre.append('Previously_Insured_x_' + i)

    fac_pro = fac_pre[1:]
    col_pro = []
    for i in fac_pro:
        df.loc[:, 'Policy_Sales_Channel_x_' + i] = pd.factorize(df['Policy_Sales_Channel'].astype(str) + df[i].astype(str))[0]
        col_pro.append('Policy_Sales_Channel_x_' + i)
    print(f"Feature engineering completed in {time.time() - start_time:.2f} seconds")
    print(f"Data shape after feature engineering: {df.shape}, Memory usage: {df.memory_usage().sum() / 1024**2:.2f} MB")
    return df, col_pre, col_pro

# Load and optimize data
train_df = import_data(train_path, index_col='id')

# Create a stratified sample of 2% from the training data
start_time = time.time()
sample_fraction = 0.02
train_sample_df, _ = train_test_split(train_df, stratify=train_df['Response'], test_size=(1 - sample_fraction), random_state=42)
print(f"Stratified sample created in {time.time() - start_time:.2f} seconds")
print(f"Sample data shape: {train_sample_df.shape}, Memory usage: {train_sample_df.memory_usage().sum() / 1024**2:.2f} MB")

# Apply feature engineering to the sampled dataset
train_sample_df, col_pre, col_pro = feature_engineering(train_sample_df)

# Split the training data into training and validation sets
start_time = time.time()
X = train_sample_df.drop('Response', axis=1)
y = train_sample_df['Response']
X_train, X_valid, y_train, y_valid = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)
print(f"Training and validation sets created in {time.time() - start_time:.2f} seconds")
print(f"Training set shape: {X_train.shape}, Memory usage: {X_train.memory_usage().sum() / 1024**2:.2f} MB")
print(f"Validation set shape: {X_valid.shape}, Memory usage: {X_valid.memory_usage().sum() / 1024**2:.2f} MB")

# Define the ColumnTransformer
start_time = time.time()
coltrans = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(sparse_output=False, dtype=np.float32), ['Gender', 'Vehicle_Damage']),
        ('minmax', MinMaxScaler(), ['Age', 'Region_Code', 'Previously_Insured', 'Policy_Sales_Channel', 'Vintage']),
        ('ordinal', OrdinalEncoder(categories=[[0, 1, 2]], dtype=np.float32), ['Vehicle_Age']),
        ('robust', RobustScaler(), ['Annual_Premium']),
        ('standard', StandardScaler(), ['Age_Type', 'Age_x_Vehicle_Age', 'Age_x_Vehicle_Damage', 'Age_x_Previously_Insured']),
        ('standard_2', StandardScaler(), col_pre + col_pro),
    ],
    remainder='passthrough'  # Keeps columns not specified in transformers
)

# Fit the transformer on the training data and transform both training and validation sets
X_train = coltrans.fit_transform(X_train)
X_valid = coltrans.transform(X_valid)
print(f"Data transformed in {time.time() - start_time:.2f} seconds")
print(f"Transformed training set shape: {X_train.shape}, Memory usage: {X_train.nbytes / 1024**2:.2f} MB")
print(f"Transformed validation set shape: {X_valid.shape}, Memory usage: {X_valid.nbytes / 1024**2:.2f} MB")

# Get feature names after transformation
feature_names = coltrans.get_feature_names_out()
gc.collect()

# Calculate the ratio for scale_pos_weight
ratio = len(train_sample_df[train_sample_df['Response'] == 0]) / len(train_sample_df[train_sample_df['Response'] == 1])
print(f"Scale pos weight ratio: {ratio}")

# Define the Optuna objective function
def objective(trial):
    params = {
        'random_state': 512,
        'objective': "binary:logistic",
        'eval_metric': 'auc',
        'max_depth': trial.suggest_int('max_depth', 7, 9),
        'min_child_weight': trial.suggest_int('min_child_weight', 12, 18),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 0.7),
        'gamma': trial.suggest_float('gamma', 0.05, 0.2),
        'learning_rate': trial.suggest_float('learning_rate', 0.1, 0.15),
        'subsample': trial.suggest_float('subsample', 0.75, 0.85),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.01, 0.3),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.7, 0.9),
        'max_bin': trial.suggest_int('max_bin', 160000, 280000),
        'scale_pos_weight': ratio,
        # 'tree_method': 'hist',
        # 'device': 'cuda',
    }
    
    num_boost_round = trial.suggest_int('num_boost_round', 2500, 3500)
    
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dvalid = xgb.DMatrix(X_valid, label=y_valid)
    
    model = xgb.train(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        evals=[(dtrain, 'train'), (dvalid, 'valid')],
        verbose_eval=100,
        early_stopping_rounds=10,
    )
    
    valid_preds = model.predict(dvalid, iteration_range=(0, model.best_iteration))
    auc_score = roc_auc_score(y_valid, valid_preds)
    
    return auc_score

# Create and run the Optuna study
print("Starting Optuna study")
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=25)

# Extract the best hyperparameters
best_params = study.best_params
best_params['random_state'] = 512
best_params['objective'] = "binary:logistic"
best_params['eval_metric'] = 'auc'
best_params['scale_pos_weight'] = ratio
best_params['tree_method'] = 'hist'
best_params['device'] = 'cuda'

# Output the best hyperparameters
print("Best hyperparameters:", best_params)


Initial memory usage: 1053.30 MB
Memory usage after optimization: 548.59 MB
Data imported and memory optimized in 7.38 seconds
Data shape: (11504798, 11), Memory usage: 548.59 MB
Stratified sample created in 7.51 seconds
Sample data shape: (230095, 11), Memory usage: 10.97 MB
Feature engineering completed in 1.17 seconds
Data shape after feature engineering: (230095, 24), Memory usage: 33.79 MB
Training and validation sets created in 0.10 seconds
Training set shape: (184076, 23), Memory usage: 26.86 MB
Validation set shape: (46019, 23), Memory usage: 6.71 MB


[I 2024-07-27 11:25:14,414] A new study created in memory with name: no-name-a8f730af-659e-42d4-9058-c0ff24ad9352


Data transformed in 0.26 seconds
Transformed training set shape: (184076, 25), Memory usage: 35.11 MB
Transformed validation set shape: (46019, 25), Memory usage: 8.78 MB
Scale pos weight ratio: 7.130278082046571
Starting Optuna study
[0]	train-auc:0.84792	valid-auc:0.84608
[100]	train-auc:0.89431	valid-auc:0.86989
[143]	train-auc:0.90244	valid-auc:0.87055


[I 2024-07-27 11:25:28,056] Trial 0 finished with value: 0.8705578606914601 and parameters: {'max_depth': 7, 'min_child_weight': 13, 'colsample_bytree': 0.6321667978993475, 'gamma': 0.07639815657030134, 'learning_rate': 0.12242598863448688, 'subsample': 0.8096162909662507, 'reg_alpha': 0.2780718793571905, 'reg_lambda': 0.8276102513823306, 'max_bin': 184854, 'num_boost_round': 3200}. Best is trial 0 with value: 0.8705578606914601.


[0]	train-auc:0.85099	valid-auc:0.84932
[100]	train-auc:0.89138	valid-auc:0.86813
[148]	train-auc:0.90106	valid-auc:0.86911


...

[0]	train-auc:0.85130	valid-auc:0.84802
[100]	train-auc:0.89843	valid-auc:0.86917
[149]	train-auc:0.91021	valid-auc:0.87044


[I 2024-07-27 11:30:17,119] Trial 23 finished with value: 0.8704662031938266 and parameters: {'max_depth': 8, 'min_child_weight': 13, 'colsample_bytree': 0.6393593105569539, 'gamma': 0.06952885919307983, 'learning_rate': 0.10062024240087251, 'subsample': 0.8321686295855483, 'reg_alpha': 0.08947279884286907, 'reg_lambda': 0.7176284196421847, 'max_bin': 248766, 'num_boost_round': 2502}. Best is trial 22 with value: 0.8706047915190843.


[0]	train-auc:0.84799	valid-auc:0.84587
[100]	train-auc:0.89235	valid-auc:0.86951
[113]	train-auc:0.89538	valid-auc:0.86958


[I 2024-07-27 11:30:29,165] Trial 24 finished with value: 0.8695955084039474 and parameters: {'max_depth': 7, 'min_child_weight': 12, 'colsample_bytree': 0.6384227330614574, 'gamma': 0.07321218933002964, 'learning_rate': 0.11377049978691271, 'subsample': 0.8258517914788286, 'reg_alpha': 0.011673791271104898, 'reg_lambda': 0.8163719052518412, 'max_bin': 275684, 'num_boost_round': 2500}. Best is trial 22 with value: 0.8706047915190843.


Best hyperparameters: {'max_depth': 8, 'min_child_weight': 13, 'colsample_bytree': 0.6392097679152726, 'gamma': 0.06286971658103871, 'learning_rate': 0.10596700150775383, 'subsample': 0.835578953888964, 'reg_alpha': 0.012132926499371949, 'reg_lambda': 0.7228323760773259, 'max_bin': 251957, 'num_boost_round': 2608, 'random_state': 512, 'objective': 'binary:logistic', 'eval_metric': 'auc', 'scale_pos_weight': 7.130278082046571, 'tree_method': 'hist', 'device': 'cuda'}


In [5]:
import optuna.visualization as vis
import plotly.io as pio

# Optimization history
opt_history = vis.plot_optimization_history(study)
pio.write_image(opt_history, 'optimization_history.png')

# Hyperparameter importance
param_importance = vis.plot_param_importances(study)
pio.write_image(param_importance, 'param_importance.png')

# Parallel coordinate plot
parallel_coord = vis.plot_parallel_coordinate(study)
pio.write_image(parallel_coord, 'parallel_coordinate.png')

# Slice plot for each hyperparameter
slice_plot = vis.plot_slice(study)
pio.write_image(slice_plot, 'slice_plot.png')

# Hyperparameter pair plot
pair_plot = vis.plot_contour(study)
pio.write_image(pair_plot, 'pair_plot.png')

